# Simple demonstration of the use of rsexecute 

In [1]:
import os
import sys

sys.path.append(os.path.join('..','..'))

import numpy

import astropy.units as u
from astropy.coordinates import SkyCoord

from rascil.data_models import PolarisationFrame

from rascil.processing_components import image_scatter_facets, image_gather_facets, create_test_image
from rascil.workflows.rsexecute.execution_support.rsexecute import rsexecute

import logging

mpl_logger = logging.getLogger("matplotlib") 
mpl_logger.setLevel(logging.WARNING) 

Set up rsexecute to use dask. If we are running in docker then the scheduler may be available by name.
Otherwise we just create a LocalCluster.

In [2]:
from dask.distributed import Client, LocalCluster
try:
    client = Client('scheduler:8786', timeout=10)
except OSError:
    client =Client(LocalCluster())
print(client)

rsexecute.set_client(client=client)


<Client: 'tcp://127.0.0.1:61915' processes=4 threads=8, memory=17.18 GB>


In [3]:
frequency = numpy.array([1e8])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')
model_graph = rsexecute.execute(create_test_image)(frequency=frequency, phasecentre=phasecentre, cellsize=0.001,
                                         polarisation_frame=PolarisationFrame('stokesI'))
model = rsexecute.compute(model_graph, sync=True)

Define a simple function to take the square root of an image

In [4]:
def imagerooter(im, **kwargs):
    im.data = numpy.sqrt(numpy.abs(im.data))
    return im

facets_graph = rsexecute.execute(image_scatter_facets, nout=16)(model_graph, facets=4)
root_graph = [rsexecute.execute(imagerooter)(facet) for facet in facets_graph]
gathered = rsexecute.execute(image_gather_facets)(root_graph, model_graph, facets=4)

Run the graph directly

In [5]:
result = rsexecute.compute(gathered, sync=True)

Check the result

In [6]:
numpy.testing.assert_array_almost_equal_nulp(result.data**2, numpy.abs(model.data), 7)

In [7]:
rsexecute.close()